# Normal Auto_FEP Workflow

## 1. 配体预处理

1. 配体"_name" 属性修改
1. 检查化学有效性
1. 依据拓扑找出最大微扰map组
1. 找出最大charge 相同组
1. 重写valid mols 到 指定文件夹

参数说明：
- `path`: 当前ligands所在目录
- `path_valid_mols`: 筛选后有效分子目录
- `path_valid_info`: 有效分子信息
- `path_invalid_info`: 无效分子信息

**Step 1. 创建工作目录，后续所有步骤的输出将写到这个目录中**

In [1]:
!mkdir -p output

**Step 2. 生成参数配置文件**

In [2]:
%%writefile output/check_ligs_input.json
{
  "path": "docs/example/lig_data",
  "path_valid_mols": "output/lig_data_output",
  "path_valid_info": "output/valid_info.csv",
  "path_invalid_info": "output/invalid_info.csv"
}

Writing output/check_ligs_input.json


**Step 3. 对指定的小分子 做配体预处理**

In [3]:
!python check_mols.py -f output/check_ligs_input.json

base-name jmc_27.mol
base-name jmc_23.mol
base-name ejm_46.mol
base-name ejm_50.mol
base-name ejm_44.mol
valid_Chemical_Rule: [<rdkit.Chem.rdchem.Mol object at 0x7f56d0c7eac0>, <rdkit.Chem.rdchem.Mol object at 0x7f56d0be3200>, <rdkit.Chem.rdchem.Mol object at 0x7f56d0be3270>, <rdkit.Chem.rdchem.Mol object at 0x7f56d0be32e0>, <rdkit.Chem.rdchem.Mol object at 0x7f56d0be3350>]
the loop is: 5
len_mol_list: (<rdkit.Chem.rdchem.Mol object at 0x7f56d0c7eac0>, <rdkit.Chem.rdchem.Mol object at 0x7f56d0be3200>, <rdkit.Chem.rdchem.Mol object at 0x7f56d0be3270>, <rdkit.Chem.rdchem.Mol object at 0x7f56d0be32e0>, <rdkit.Chem.rdchem.Mol object at 0x7f56d0be3350>)
matches ((0, 1, 2, 3, 4, 5, 28, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 32, 16, 17, 18, 33, 31, 30, 29, 23, 27, 26),)
c1([H])c([H])c(Cl)c(C(=O)N(c2c([H])c([H])nc(N(C=O)[H])c2[H])[H])c(Cl)c1[H]
the loop is: 4


## 2. 生成微扰 map

edge 属性：
1. perturbed_atoms: 微扰原子数
1. desirablility: 边的可信度(默认为1)
1. atom_map: 分子的原子匹配关系（需要3D可视化）
1. cost: 计算该pair的代价；与微扰原子数相关
1. similarity: 分子相似性
1. MCS: pair 共有子结构

参数说明：
- `input`: ligands所在位置
- `progress_file`: 输出的 plk 文件
- `map_info_file`: 输出的 json 文件表示的微扰图结构
- `graph_file`: 输出的 svg 文件表示的微扰图结构
- `map_type`: 微扰map 类型
- `map_bias`: 微扰map 中心分子

**Step 1. 生成参数配置文件**

In [4]:
%%writefile output/gen_map.ini
[generate_perturbation_map]

input = output/lig_data_output
progress_file = output/progress.pkl
map_info_file = output/generdated_map_info.json
graph_file = output/best_graph.svg
map_type = star
map_bias = ejm_46

Writing output/gen_map.ini


**Step 2. 生成微扰图**

In [5]:
!python generate_perturbation_map_atom_map_wangwei.py --config_file output/gen_map.ini

the inputs are output/lig_data_output
the file is output/lig_data_output/jmc_27.mol
the file is output/lig_data_output/jmc_23.mol
the file is output/lig_data_output/ejm_46.mol
the file is output/lig_data_output/ejm_50.mol
the file is output/lig_data_output/ejm_44.mol
all_mols: OrderedDict([('jmc_27', <rdkit.Chem.rdchem.Mol object at 0x7f861a5f9120>), ('jmc_23', <rdkit.Chem.rdchem.Mol object at 0x7f861a620ba0>), ('ejm_46', <rdkit.Chem.rdchem.Mol object at 0x7f861a620c10>), ('ejm_50', <rdkit.Chem.rdchem.Mol object at 0x7f861a620c80>), ('ejm_44', <rdkit.Chem.rdchem.Mol object at 0x7f861a620b30>)])
map_bias: ejm_46
full_thermograph: DiGraph with 5 nodes and 4 edges
pos: {'jmc_27': array([9.99999986e-01, 2.18556937e-08]), 'jmc_23': array([-3.57647606e-08,  1.00000000e+00]), 'ejm_50': array([-9.9999997e-01, -6.5567081e-08]), 'ejm_44': array([ 1.98715071e-08, -9.99999956e-01]), 'ejm_46': [0.0, 0.0]}


## 3. 修改微扰map

根据对微扰图中边的增删配置，修改 progress.pkl 文件，同时更新 map_info 文件 和 链接图。
修改时会同时删除 度为0的结点。

参数说明：
- `progress_file`: 被修改的 plk 文件
- `map_info_file`: 输出的 json 文件表示的微扰图结构
- `graph_file`: 输出的 svg 文件表示的微扰图结构

**Step 1. 生成参数配置文件**

In [7]:
%%writefile output/modify_map.ini
[modify_perturbation_map]

progress_file = output/progress.pkl
map_info_file = output/generdated_map_info.json
graph_file = output/best_graph.svg

Writing output/modify_map.ini


**Step 2. 生成修改配置文件**

In [9]:
%%writefile output/modify_map_input.json
{
  "add_edge": [
    [
      "jmc_23",
      "ejm_46"
    ]
  ],
  "del_edge": [
    [
      "ejm_46",
      "jmc_23"
    ]
  ]
}

Writing output/modify_map_input.json


**Step 3. 调用接口，根据参数修改微扰图**

In [10]:
!python modify_perturbation_map_zcc.py --config_file output/modify_map.ini -f output/modify_map_input.json

mcs:  ['jmc_23', 'ejm_46'] [H]c1nc(N([H])C(=O)[C@]2([H])C([H])C2([H])[H])c([H])c(N([H])C(=O)c2c(Cl)c([H])c([H])c([H])c2Cl)c1[H]
add_edge_mcs:  [['jmc_23', 'ejm_46']]
molecules_dict_mol:  {'jmc_27': <rdkit.Chem.PropertyMol.PropertyMol object at 0x7fdd331e4310>, 'jmc_23': <rdkit.Chem.PropertyMol.PropertyMol object at 0x7fdd331e4400>, 'ejm_46': <rdkit.Chem.PropertyMol.PropertyMol object at 0x7fdd331e44a0>, 'ejm_50': <rdkit.Chem.PropertyMol.PropertyMol object at 0x7fdd331e45e0>, 'ejm_44': <rdkit.Chem.PropertyMol.PropertyMol object at 0x7fdd331e46d0>}
perturbation_graph_edges: [('ejm_46', 'jmc_27', {'perturbed_atoms': 1, 'desirability': 1.0, 'mcs': <rdkit.Chem.rdchem.Mol object at 0x7fdd331e4c20>, 'atom_map': [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 

## 4. 修改指定边的AtomMap

根据对 Atom Map 中边的增删配置，修改 progress.pkl 文件，同时更新 map_info 文件。

参数说明：
- `progress_file`: 被修改的 plk 文件
- `map_info_file`: 输出的 json 文件表示的微扰图结构

**Step 1. 生成参数配置文件**

In [12]:
%%writefile output/modify_atom_map.ini
[modify_atom_map]

progress_file = output/progress.pkl
map_info_file = output/generdated_map_info.json

Overwriting output/modify_atom_map.ini


**Step 2. 生成修改配置文件**

In [18]:
%%writefile output/modify_atom_map_input.json
{
  "add_atom_pair": {
    "ejm_46,jmc_27": [
      [
        1,
        1
      ]
    ]
  },
  "del_atom_pair": {
    "ejm_46,jmc_27": [
      [
        0,
        0
      ],
      [
        1,
        1
      ]
    ]
  }
}

Overwriting output/modify_atom_map_input.json


**Step 3. 调用接口，根据参数修改原子图**

In [19]:
!python modified_lig_atommap.py --config_file output/modify_atom_map.ini -f output/modify_atom_map_input.json

('ejm_46', 'jmc_27') [(2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (10, 10), (11, 11), (12, 12), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (19, 19), (20, 20), (21, 21), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (32, 32), (33, 33), (34, 34), (35, 35)] 3
finished modified_lig_atommap


## 5. align 配体到口袋

将所有配体的构象匹配到口袋，首先将 中心分子 基于Docking方法对接到口袋，这个方法可以把其他的配体分子对齐到中心分子，这样也就匹配到口袋了。

参数说明：
- `progress_file`: 参考pkl文件
- `ref_mol_path`: 经过 docking 的分子作为参考分子
- `lig_data_path`: 待匹配的分子

**Step 1. 生成参数文件**

In [24]:
%%writefile output/align_ligs_input.json
{
  "progress_file": "output/progress.pkl",
  "ref_mol_path": "docs/example/lig_data/ejm_46.mol",
  "lig_data_path": "output/lig_data_output",
  "aligned_lig_data_path": "output/aligned_lig_data"
}

Overwriting output/align_ligs_input.json


**Step 2. 调用接口，对齐配体到口袋**

In [26]:
!python alignment_ligands_zcc.py -f output/align_ligs_input.json

translation_list (8, 7, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 15, 3, 2, 23, 1, 0, 5, 4, 6)
translation_list (0, 1, 2, 23, 3, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 15, 4, 6, 5)
translation_list (8, 7, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 15, 3, 2, 23, 1, 0, 5, 4, 6)
translation_list (0, 1, 2, 20, 3, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 15, 4, 6, 5)
translation_list (8, 7, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 15, 3, 2, 23, 1, 0, 5, 4, 6)
translation_list (0, 1, 2, 20, 3, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 15, 4, 6, 5)
translation_list (0, 1, 2, 23, 3, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 15, 4, 6, 5)
translation_list (8, 7, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 15, 3, 2, 23, 1, 0, 5, 4, 6)


## 6. 生成配体拓扑文件(.itp)

为配体分子生成拓扑文件，目前有两种方法：
- sob: 基于sobtop 软件生成 配体力场，特点：生成拓扑文件速度比较高；缺点，生成带电荷配体力场会出错，且不能生成gaff2力场
- acp: acpype软件优点是，配体无论带有电荷均可以生成拓扑文件，并且能够生成gaff 和 gaff2两种版本的gaff力场拓扑文件；缺点：生成拓扑文件速率较低

In [ ]:
# 使用sobtop生成拓扑文件
!python gen_lig_itp_sob.py -input /home/work/pyautofep/PyAutoFEP/aligned_lig_data

In [ ]:
# 使用acpype生成拓扑文件
!python gen_lig_itp_acp.py --lig_dir output/aligned_lig_data --output_dir output/lig_itp_data

## 7. 生成dual_topl文件

配置文件参数说明：
- `input_ligands`      : ligands 所在目录
- `structure`          : protein 所在目录
- `perturbations_dir`  : 生成文件所在目录
- `gmx_bin_local`      : gromacs 命令
- `buildsys_forcefield`: protein 采用力场 1: amber03
- `md_temperature`     : MD模拟温度:*k
- `md_length`          : MD模拟时间长度：5000 ps

**Step 1. 生成参数文件**

In [ ]:
%%writefile output/step2.ini
[prepare_dual_topology]

input_ligands = output/aligned_lig_data
structure = docs/example/5q17_processed.pdb
perturbations_dir = tutorial
gmx_bin_local = gmx_mpi
buildsys_forcefield = 1
md_temperature = 298.15
md_length = 5000.0 

**Step 2. 调用接口，生成dual topl文件**

In [ ]:
!python prepare_dual_topology_zcc.py --config_file=step2.ini --output_hidden_temp_dir False